In [1]:
import time
import datetime
import numpy as np
import pandas as pd
import os

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
print(time.strftime("%Y-%m-%d %H:%M:%S"))

2018-05-01 18:31:36


In [4]:
class Timer():
    def __init__(self):
        self.info = 'main'
        self.start_time = time.time()
    
    def start(self, info):
        self.info = info
        self.start_time = time.time()
        self.checkpoint('start', elapsed_on=False)
    
    def end(self):
        self.checkpoint(' end ')
        
    def checkpoint(self, tag, elapsed_on=True):
        if elapsed_on:
            elapsed = datetime.timedelta(seconds=round(time.time() - self.start_time))
            expanded_info = self.info + ' [time elapsed: %s]' % str(elapsed)
        else:
            expanded_info = self.info
        self.output(tag, info=expanded_info)
        
    def output(self, tag=' '*5, info=''):
        if type(info) != type(''):
            info = str(info)
        print('[%s] (%s) %s' % (Timer.get_current_time(), tag, info))
    
    @staticmethod
    def get_current_time():
        return time.strftime("%Y-%m-%d %H:%M:%S")

timer = Timer()
sub_timer = Timer()

# Load Data

In [5]:
timer.start('Load Data')
# directory = '../data/split/'
# df_train = pd.read_csv(directory + 'train.csv')
# df_test_warm = pd.read_csv(directory + 'test_warm.csv')
# df_test_cold_user = pd.read_csv(directory + 'test_cold_user.csv')
# df_test_cold_item = pd.read_csv(directory + 'test_cold_item.csv')

[2018-05-01 18:31:36] (start) Load Data


In [6]:
directory = '../data/context/'
df_event_context = pd.read_csv(directory + 'event_context.csv')
df_song_context = pd.read_csv(directory + 'song_context.csv')
df_user_context = pd.read_csv(directory + 'user_context.csv')
timer.checkpoint('context')

[2018-05-01 18:31:43] (context) Load Data [time elapsed: 0:00:07]


In [7]:
num_user = len(df_user_context.user_id.unique())
num_item = len(df_song_context.song_id.unique())
print (num_user)
print (num_item)

30755
359966


In [8]:
# # load target sets
# import pickle
# with open('../data/split/target_set.pickle', 'rb') as handle:
#     target_set = pickle.load(handle)
# with open('../data/split/train_target_set.pickle', 'rb') as handle:
#     train_target_set = pickle.load(handle)
# with open('../data/split/test_warm_target_set.pickle', 'rb') as handle:
#     test_warm_target_set = pickle.load(handle)
# with open('../data/split/test_cold_user_target_set.pickle', 'rb') as handle:
#     test_cold_user_target_set = pickle.load(handle)
# with open('../data/split/test_cold_item_target_set.pickle', 'rb') as handle:
#     test_cold_item_target_set = pickle.load(handle)

In [9]:
class Data():
    def __init__(self, name):
        '''
        user_list: list(int), the list of user id's used in the dataset
        target_set: list(set), set of target items for each user
        item_list: list(numpy array), list of items used in the dataset for each user
        '''
        self.name = name
        self.df = None
        self.user_list = None
        self.item_list = None
        self.target_set = None
    
    def load(self, filename):
        self.df = pd.read_csv(filename)
        # prepare user list
        self.user_list = self.df['user_id'].unique()
        
        # prepare item list
        self.item_list = [[] for i in range(num_user)]
        self.df.apply(
            lambda row: self.item_list[row['user_id']].append(row['song_id']),
            axis=1
        )
        self.item_list = list(map(np.array, self.item_list))
        
        # prepare target set
        self.target_set = [set() for i in range(num_user)]
        self.df[self.df['target'] == 1].apply(
            lambda row: self.target_set[row['user_id']].add(row['song_id']),
            axis=1
        )
        
# def load_split(name):
#     data = Data(name)
#     # load the user ids in the data set
#     with open('../data/split/' + name + '_user_list.pickle', 'rb') as handle:
#         data.user_list = pickle.load(handle)
        
#     # load the list(set) for items in the data set with label=1
#     with open('../data/split/' + name + '_target_set.pickle', 'rb') as handle:
#         data.target_set = pickle.load(handle)
        
#     # load the list(set) for all items in the data set
#     with open('../data/split/' + name + '_item_set.pickle', 'rb') as handle:
#         data.item_set = pickle.load(handle)
        
#     return data

def load_split(name):
    directory = '../data/split/'
    data = Data(name)
    data.load(directory + name + '.csv')
    return data

In [10]:
# data_train = load_split('train')
# data_test_warm = load_split('test_warm')
# data_test_cold_user = load_split('test_cold_user')
# data_test_cold_item = load_split('test_cold_item')

data_train = load_split('train')
data_test_warm = load_split('test_warm')
data_test_cold_user = load_split('test_cold_user')
data_test_cold_item = load_split('test_cold_item')
timer.end()

[2018-05-01 18:35:05] ( end ) Load Data [time elapsed: 0:03:29]


In [11]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Reshape, Lambda
from keras.layers.merge import concatenate, dot
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.initializers import RandomUniform, RandomNormal, TruncatedNormal, Zeros
from keras.optimizers import RMSprop, Adam, SGD
from sklearn.metrics import mean_squared_error
import tensorflow as tf

Using TensorFlow backend.


In [12]:
REG_LAMBDA = 0
EMBED_DIM = 64

vocab_size = num_user
user_embeddings = Embedding(
    input_dim = vocab_size,
    output_dim = EMBED_DIM,
    embeddings_initializer = RandomUniform(minval=-0.1, maxval=0.1),
    embeddings_regularizer = l2(REG_LAMBDA),
    input_length = 1,
    name = 'user_embed',
    trainable=True)

vocab_size = num_item
item_embeddings = Embedding(
    input_dim = vocab_size,
    output_dim = EMBED_DIM,
    embeddings_initializer = RandomUniform(minval=-0.1, maxval=0.1),
    embeddings_regularizer=l2(REG_LAMBDA),
    input_length=1,
    name = 'item_embed',
    trainable=True)

# embedding of user id
uid_input = Input(shape=(1,), dtype='int32')
embedded_user = user_embeddings(uid_input)
embedded_user = Reshape((EMBED_DIM,))(embedded_user)

# embedding of song id
iid_input = Input(shape=(1,), dtype='int32')
embedded_item = item_embeddings(iid_input)
embedded_item = Reshape((EMBED_DIM,))(embedded_item)

# dot production of embedded vectors
preds = dot([embedded_user, embedded_item], axes=1, name='dot_score')

# embedding model
user_embed_model = Model(inputs=uid_input, outputs=embedded_user)
item_embed_model = Model(inputs=iid_input, outputs=embedded_item)

model_MF = Model(inputs=[uid_input, iid_input], outputs=preds)
model_MF.compile(
    loss=keras.losses.mean_squared_error, 
    optimizer=RMSprop(lr=1e-3),
#     optimizer=SGD(lr=1e-4),
    metrics=[keras.metrics.mean_squared_error])

In [13]:
model_directory = '../model/mf/'
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
model_path = model_directory + 'mf_model.h5'

In [14]:
dropout_model_path = '../model/dropout/model.h5'

In [15]:
# load the best model
model_MF.load_weights(model_path)

In [16]:
def single_top_k(score_list, k):
    ind = np.argpartition(score_list, -k)[-k:]
    top_k_ind = list(reversed(ind[np.argsort(score_list[ind])]))
    return np.array(top_k_ind)

# try to implement a two-dimensional top_k
def two_dim_top_k(a, k):
    return np.array([single_top_k(row, k) for row in a])

def top_k(a, k):
    if len(a.shape) == 1:
        return single_top_k(a, k)
    elif len(a.shape) == 2:
        return two_dim_top_k(a, k)
    else:
        return None

In [17]:
user = 1
item_list = np.array([1,2,3,4])
v_user = user_embed_model.predict(np.array([1, 2]))
v_item = item_embed_model.predict(item_list)

print(v_user.shape)

#_x = v_user @ v_item.T #require python 3.5
_x = np.matmul(v_user, v_item.T)
print(_x)

print(model_MF.predict([
    np.repeat(user, len(item_list)),
    item_list
]).flatten())

(2, 64)
[[  2.59375989e-01  -3.29650007e-02   4.17865440e-02   1.46565601e-01]
 [ -6.12905342e-03   2.17780974e-02  -5.20521775e-03  -2.27784272e-04]]
[ 0.25937599 -0.032965    0.04178654  0.1465656 ]


In [18]:
print(len(data_test_warm.item_list[1]), len(data_test_warm.target_set[1]))

(41, 19)


In [19]:
# recall at k
sess = tf.Session()
v_user_all = user_embed_model.predict(np.arange(num_user))
v_item_all = item_embed_model.predict(np.arange(num_item))
    
def __recall(klist, target, recommend_list):
    den = len(target) # denominator
    recall_value = 0.0
    recall_list = []
    for k in klist:
        if den < k:
            recall_value = 1.0
        if recall_value == 1.0: # if it's already 1.0, it should be 1.0 after
            recall_list.append(recall_value)
            continue
        recommend_set = set(recommend_list[:k])
        num = len(target & recommend_set)
        recall_value = float(num) / float(den)
        recall_list.append(recall_value)
    return recall_list


def recall_mf(model, klist, data):
    '''
    :param klist: the list of k's in recall@k, e.g. [50, 100, 150, ...]
    :param data: data set for evaluation
        - user_list
        - target_set
        - item_set
    :return: list(float) for recall at each k, with the same size as klist
    '''
    recall_at_k = []
    max_k = max(klist)
    t1, t2, t3, t4, t5 = 0, 0, 0, 0, 0
    for user in data.user_list:
        # get the corresponding embedded vectors
        v_user = v_user_all[user]
        v_item = v_item_all[data.item_list[user]]
        
        # compute the scores
        #score_list = v_user @ v_item.T
        score_list = np.matmul(v_user, v_item.T)
        score_list = score_list.flatten()
        # assert len(score_list) == len(data.item_list[user])
        
        k = min(max_k, len(data.item_list[user]))
        # get the recommended list
        indices = top_k(score_list, k)
        recommend_list = data.item_list[user][indices]
        
        # evaluate recall
        recall_at_k.append(__recall(klist, data.target_set[user], recommend_list))
    return np.mean(recall_at_k, axis=0)


def recall_random(klist, data):
    recall_at_k = []
    max_k = max(klist)
    for i, user in enumerate(data.user_list):
        # compute the scores
        score_list = np.random.uniform(low=0, high=1, size=len(data.item_list[user]))
        
        k = min(max_k, len(data.item_list[user]))
        indices = top_k(score_list, k)
        recommend_list = data.item_list[user][indices]
        
        # evaluate recall
        recall_at_k.append(__recall(klist, data.target_set[user], recommend_list))
    return np.mean(recall_at_k, axis=0)

The following code will be quite slow: each user takes around 4s to run.

```
score_list = model_MF.predict([
    np.repeat(user, len(item_list)),
    item_list
]).flatten()
```

**It's because fetching embeddings are slow.** So get the embedding first and use matrix multiplication!

After modification, it will still take > 30 mintues (I don't know how long...)

### Note!!

It's **far more** better to get more from the model at one time rather than calling the model multiple times!

The difference between warm and cold is due to the difference in denominator (maybe), so don't compare them with each other.

## TODO: Dropout Net

### Define and initialize the model

In [20]:
user_CATEGORICAL = [
    'city', 'gender', 'registered_via', 'registration_year', 
    'registration_month', 'registration_day', 'expiration_year', 
    'expiration_month', 'expiration_day']
user_NUMERICAL = ['age', 'weird_age', 'validate_days']
set(df_user_context.columns) - (set(user_CATEGORICAL).union(set(user_NUMERICAL))), \
set(user_CATEGORICAL).intersection(set(user_NUMERICAL))

({'Unnamed: 0', 'user_id'}, set())

In [21]:
item_CATEGORICAL = [
    'artist_name', 'composer', 'genre_ids', 'language', 
    'lyricist', 'song_year']
item_NUMERICAL = [
    'song_length', 'genre_count', 'lyricist_count',
    'composer_count', 'artist_count', 'is_featured',
    'artist_composer', 'artist_composer_lyricist', 
    'song_lang_boolean', 'smaller_song']
set(df_song_context.columns) - (set(item_CATEGORICAL).union(set(item_NUMERICAL))), \
set(item_CATEGORICAL).intersection(set(item_NUMERICAL))

({'Unnamed: 0', 'song_id'}, set())

In [22]:
def evaluate_cold(x, recall_at):
    embedding_prod_cold = tf.matual(x[0], x[1], transpose_b = True, name='pred_all_items')
    _, eval_preds_cold = tf.nn.top_k(embedding_prod_cold, k=recall_at[-1], sorted=True, name='topK_net_cold')
    return eval_preds_cold

def evaluate_warm(x, recall_at):
    embedding_prod_cold = tf.matual(x[0], x[1], transpose_b = True)
    embedding_prod_warm = tf.sparse_add(embedding_prod_cold, x[2])
    _, eval_preds_warm = tf.nn.top_k(embedding_prod_warm, k=recall_at[-1], sorted=True, name='topK_net_warm')
    return eval_preds_warm

def prediction(x):
    return tf.matmul(x[0], x[1], transpose_b=True)

def topk_vals(x, num_candidates):
    tf_topk_vals, _ = tf.nn.top_k(x, k=num_candidates, sorted=True)
    return tf.reshape(tf_topk_vals, [-1], name='select_y_vals')

def topk_inds(x, num_candidates):
    _, tf_topk_inds = tf.nn.top_k(x, k=num_candidates, sorted=True)
    return tf.reshape(tf_topk_inds, [-1], name='select_y_vals')

def random_target(x, num_candidates):
    preds_random = tf.gather_nd(x[0], x[1])
    return tf.reshape(preds_random, [-1], name='random_y_inds')

def latent_topk_cold(x, recall_at):
    _, tf_latent_topk_cold = tf.nn.top_k(x, k=recall_at[-1], sorted=True, name='topK_latent_cold')
    return tf_latent_topk_cold

def latent_topk_warm(x, recall_at):
    preds_pref_latent_warm = tf.sparse_add(x[0], x[1])
    _, tf_latent_topk_warm = tf.nn.top_k(preds_pref_latent_warm, k=recall_at[-1], sorted=True, name='topK_latent_warm')
    return tf_latent_topk_warm

def dense_batch_fc_tanh(x, units, scope, do_norm=False):
#     w_init = tf.truncated_normal_initializer(stddev=0.01)
#     b_init = tf.zeros_initializer()
#     h1 = Dense(units, kernel_initializer = w_init, bias_initializer = b_init)(x)
    h1 = Dense(units, kernel_initializer = TruncatedNormal(stddev=0.01), bias_initializer = Zeros())(x)
    if do_norm:
        # h2 = BatchNormalization(momentum = 0.9, center=True, scale=True, training=phase)(h1)
        h2 = BatchNormalization(momentum = 0.9, center=True, scale=True)(h1)
        return Activation('tanh')(h2)
    else:
        return Activation('tanh')(h1)

In [47]:
class DeepCF:
    """
    main model class implementing DeepCF
    also stores states for fast candidate generation

    latent_rank_in: rank of preference model input
    user_content_rank: rank of user content input
    item_content_rank: rank of item content input
    model_select: array of number of hidden unit,
        i.e. [200,100] indicate two hidden layer with 200 units followed by 100 units
    rank_out: rank of latent model output

    """

    def __init__(self, latent_rank_in, model_select, rank_out):
        self.rank_in = latent_rank_in
        self.model_select = model_select
        self.rank_out = rank_out

    def context_model(self, tag, df_context, CATEGORICAL, NUMERICAL):
        input_layers = []
        embed_layers = []
        for col in CATEGORICAL:
            input_layer = Input(shape=(1,), name=tag + '_' + col + '_input')
            input_layers.append(input_layer)
            vocab_size = df_context[col].max() + 1
            embed_size = np.power(2, int(np.ceil(np.log2(np.log2(vocab_size)))))
            print('[%s] %-20s\tvocab: %-8d, embed: %-4d' % (tag, col, vocab_size, embed_size))
            embed_layer = Embedding(
                input_dim = vocab_size,
                output_dim = embed_size,
                embeddings_initializer = RandomUniform(minval=-0.1, maxval=0.1),
                embeddings_regularizer = l2(1e-4),
                input_length = 1,
                name = tag + '_' + col+'_embed',
                trainable=True)
            embed_layer = embed_layer(input_layer)
            embed_layer = Reshape((embed_size,))(embed_layer)
            embed_layers.append(embed_layer)
            
        numerical_input = Input(shape=(len(NUMERICAL),), name=tag+'_numerical_input')
        input_layers.append(numerical_input)
        
        preds = concatenate(embed_layers + [numerical_input], name=tag + '_content')
#         preds = Dense(64, activation='relu', name=tag + '_content_dense1')(preds)
#         preds = Dropout(0.5, name=tag + '_content_dropout')(preds)
#         preds = Dense(64, name=tag + '_content_dense1')(preds)
        return input_layers, preds
            
    def build_model(self):
        self.Vin = Input(shape=(self.rank_in,), dtype='float32', name='V_in_raw')
        self.Uin = Input(shape=(self.rank_in,), dtype='float32', name='U_in_raw')
        
        self.user_inputs, self.Ucontent = self.context_model(
            'user', df_user_context, CATEGORICAL=user_CATEGORICAL, NUMERICAL=user_NUMERICAL)
        self.item_inputs, self.Vcontent = self.context_model(
            'item', df_song_context, CATEGORICAL=item_CATEGORICAL, NUMERICAL=item_NUMERICAL)
        
        u_concat = concatenate([self.Uin, self.Ucontent])
        v_concat = concatenate([self.Vin, self.Vcontent])
        u_last = u_concat
        v_last = v_concat
        for ihid, hid in enumerate(self.model_select):
            u_last = dense_batch_fc_tanh(u_last, hid, 'user_layer_%d' % (ihid + 1), do_norm=True)
            v_last = dense_batch_fc_tanh(v_last, hid, 'item_layer_%d' % (ihid + 1), do_norm=True)

        self.U_embedding = Dense(
            self.rank_out, 
            kernel_initializer = TruncatedNormal(stddev=0.01),
            bias_initializer = Zeros())(u_last)
        self.V_embedding = Dense(
            self.rank_out, 
            kernel_initializer = TruncatedNormal(stddev=0.01),
            bias_initializer = Zeros())(v_last)
        self.preds = dot([self.U_embedding, self.V_embedding], axes=1, name='dot_score')
        self.input_layers = [self.Uin, self.Vin] + self.user_inputs + self.item_inputs
        
        self.user_model = Model(inputs=[self.Uin] + self.user_inputs, outputs=self.U_embedding)
        self.item_model = Model(inputs=[self.Vin] + self.item_inputs, outputs=self.V_embedding)
        
        model = Model(inputs=self.input_layers, outputs=self.preds)
        # optimizer = RMSprop()
        optimizer = RMSprop(lr=1e-2)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        self.model = model

In [48]:
latent_rank_in = 64
model_select = [200, 100]
rank_out = 100

dropout_net = DeepCF(latent_rank_in, model_select, rank_out)
dropout_net.build_model()
dropout_net.model.summary()

[user] city                	vocab: 21      , embed: 8   
[user] gender              	vocab: 3       , embed: 2   
[user] registered_via      	vocab: 5       , embed: 4   
[user] registration_year   	vocab: 14      , embed: 4   
[user] registration_month  	vocab: 12      , embed: 4   
[user] registration_day    	vocab: 31      , embed: 8   
[user] expiration_year     	vocab: 18      , embed: 8   
[user] expiration_month    	vocab: 12      , embed: 4   
[user] expiration_day      	vocab: 31      , embed: 8   
[item] artist_name         	vocab: 40583   , embed: 16  
[item] composer            	vocab: 76064   , embed: 32  
[item] genre_ids           	vocab: 573     , embed: 16  
[item] language            	vocab: 10      , embed: 4   
[item] lyricist            	vocab: 33888   , embed: 16  
[item] song_year           	vocab: 100     , embed: 8   
__________________________________________________________________________________________________
Layer (type)                    Output Shape  

### Help Functions

In [25]:
class Stopwatch():
    def __init__(self, info=''):
        self.total = 0
        self.info = info
    
    def clear(self):
        self.total = 0
    
    def tic(self):
        self.start_time = time.time()
    
    def toc(self):
        self.total += time.time() - self.start_time
    
    def show(self):
        print('%.3f seconds \t %s' % (self.total, self.info))

In [26]:
def generate_user_content(user_list):
    return [df_user_context.loc[user_list, col] for col in user_CATEGORICAL] + [df_user_context.loc[user_list, user_NUMERICAL]]
        
def generate_item_content(item_list):
    return [df_song_context.loc[item_list, col] for col in item_CATEGORICAL] + [df_song_context.loc[item_list, item_NUMERICAL]]

In [40]:
def numpy_array_flat_join(list_of_list):
    ret = []
    for l in list_of_list:
        for e in l:
            ret.append(e)
    return np.array(ret)

### Configuration

In [28]:
# configuration
user_batch_size = 1000
# n_scores_user = 2500
n_scores_user = 100

data_batch_size = 100
max_data_per_step = 2500000
dropout = 0.5

# counting variables
n_step = 0
n_batch_trained = 0

### Preprocessing

In [27]:
import utils

u_pref = v_user_all
v_pref = v_item_all
_, u_pref_scaled = utils.prep_standardize(u_pref)
_, v_pref_scaled = utils.prep_standardize(v_pref)
v_pref_expanded = np.vstack([v_pref_scaled, np.zeros_like(v_pref_scaled[0, :])])
v_pref_last = v_pref_scaled.shape[0] # the last v_pref_scaled TODO: maybe a all zero?
u_pref_expanded = np.vstack([u_pref_scaled, np.zeros_like(u_pref_scaled[0, :])])
u_pref_last = u_pref_scaled.shape[0]

In [29]:
# get top k matrix, the i-th row is the top k items for the i-th user
stopwatch = Stopwatch()
stopwatch.tic()
topK_matrix = [top_k(np.matmul(u_pref[i], v_pref.T), k = n_scores_user) for i in range(num_user)]
topK_matrix = np.array(topK_matrix)
stopwatch.toc()
stopwatch.show() # around 600s = 10min

650.950 seconds 	 


### Real Training

In [53]:
timer.start('training')
# profiling
sw = [Stopwatch() for _ in range(20)]

num_epoch = 10
row_index = np.copy(data_train.user_list)
for epoch in range(num_epoch):
    sw[0].tic()
    np.random.shuffle(row_index)
    sw[0].toc()
    # generate user batches
    whole_target_scores_list = []
    whole_target_items_list = []
    whole_target_users_list = []
    for b in utils.batch(row_index, user_batch_size):
        # prep targets
        # For each user, n_scores_user of the top (defined by the scores of 
        # dot product of WMF latent vectors) items are included in the batch
        # Besides that, n_scores_user of random items with their scores
        # (calculated as above) are also included in the batch, which means
        # the ratio is 1:1 for topN and randomN
        
        # get top_N items for the users
        # NOTE: if this takes long time, we can preprocess this part of data and store them
        sw[1].tic()
        target_users = np.repeat(b, n_scores_user)
        # target_items = top_k(score_matrix, k = n_scores_user).flatten()
        target_items = topK_matrix[b].flatten()
        score_matrix = np.matmul(u_pref[b, :], v_pref.T)
        sw[1].toc()
        
        # get random_N
        sw[2].tic()
        target_users_rand = np.repeat(b, n_scores_user)
        target_items_rand = [np.random.choice(num_item, n_scores_user) for _ in b]
        target_items_rand = np.array(target_items_rand).flatten()
        sw[2].toc()
        
        sw[3].tic()
        target_scores = score_matrix[:, target_items]
        random_scores = score_matrix[:, target_items_rand]
        sw[3].toc()

        # merge topN and randomN items per user
        sw[4].tic()
        target_users = np.append(target_users, target_users)
        target_items = np.append(target_items, target_items_rand)
        target_scores = np.append(target_scores, random_scores)
        # append to the epoch list
        whole_target_users_list.append(target_users)
        whole_target_items_list.append(target_items)
        whole_target_scores_list.append(target_scores)
        sw[4].toc()
    
    # joining the user batches into a epoch data
    sw[5].tic()
    target_users = numpy_array_flat_join(whole_target_users_list)
    target_items = numpy_array_flat_join(whole_target_items_list)
    target_scores = numpy_array_flat_join(whole_target_scores_list)
    sw[5].toc()

    # generate content (should before dropout)
    sw[6].tic()
    user_content = generate_user_content(target_users)
    item_content = generate_item_content(target_items)
    sw[6].toc()
    
    # dropout process
    sw[7].tic()
    n_targets = len(target_scores)
    # n_targets = min(n_targets, max_data_per_step)
    n_to_drop = int(np.floor(dropout * n_targets))
    user_to_drop = np.random.choice(np.arange(n_targets), size=n_to_drop, replace=False)
    item_to_drop = np.random.choice(np.arange(n_targets), size=n_to_drop, replace=False)
    target_users[user_to_drop] = u_pref_last
    target_items[item_to_drop] = v_pref_last
    sw[7].toc()
            
    sw[8].tic()
    loss_out = dropout_net.model.fit(
        x = [u_pref_expanded[target_users, :], v_pref_expanded[target_items, :]] + user_content + item_content,
        y = target_scores, # target
        epochs = 1,
        batch_size = data_batch_size,
        shuffle = True
    )
    sw[8].toc()
    dropout_model_path = '../model/dropout/exp2/model_{}.h5'.format(epoch)
    dropout_net.model.save_weights(dropout_model_path)
timer.end()

[2018-05-01 22:32:43] (start) training
(1000, 100000)
(1000, 100000)
(1000,)


Exception: 

In [ ]:
for i in range(9):
    print i,
    sw[i].show()

In [33]:
# num_epoch = 10
# row_index = np.copy(data_train.user_list)
# for epoch in range(num_epoch):
#     sw[0].tic()
#     np.random.shuffle(row_index)
#     sw[0].toc()
#     # generate user batches
#     for b in utils.batch(row_index, user_batch_size):
#         # n_step: number of batch
#         n_step += 1
#         # prep targets
#         # For each user, n_scores_user of the top (defined by the scores of 
#         # dot product of WMF latent vectors) items are included in the batch
#         # Besides that, n_scores_user of random items with their scores
#         # (calculated as above) are also included in the batch, which means
#         # the ratio is 1:1 for topN and randomN
        
#         # get top_N items for the users
#         # NOTE: if this takes long time, we can preprocess this part of data and store them
#         sw[1].tic()
#         target_users = np.repeat(b, n_scores_user)
#         # target_items = top_k(score_matrix, k = n_scores_user).flatten()
#         target_items = topK_matrix[b].flatten()
#         score_matrix = np.matmul(u_pref[b, :], v_pref.T)
#         sw[1].toc()
        
#         # get random_N
#         sw[2].tic()
#         target_users_rand = np.repeat(np.arange(len(b)), n_scores_user)
#         target_items_rand = [np.random.choice(v_pref.shape[0], n_scores_user) for _ in b]
#         target_items_rand = np.array(target_items_rand).flatten()
#         sw[2].toc()
        
#         sw[3].tic()
#         target_scores = score_matrix[target_users_rand, target_items]
#         random_scores = score_matrix[target_users_rand, target_items_rand]
#         sw[3].toc()

#         # merge topN and randomN items per user
#         sw[4].tic()
#         target_scores = np.append(target_scores, random_scores)
#         target_items = np.append(target_items, target_items_rand)
#         target_users = np.append(target_users, target_users)
#         sw[4].toc()

#         # generate content (should before dropout)
#         sw[5].tic()
#         user_content = generate_user_content(target_users)
#         item_content = generate_item_content(target_items)
#         sw[5].toc()
        
#         # dropout process
#         sw[6].tic()
#         n_targets = len(target_scores)
#         # n_targets = min(n_targets, max_data_per_step)
#         n_to_drop = int(np.floor(dropout * n_targets))
#         user_to_drop = np.random.choice(np.arange(n_targets), size=n_to_drop, replace=False)
#         item_to_drop = np.random.choice(np.arange(n_targets), size=n_to_drop, replace=False)
#         target_users[user_to_drop] = u_pref_last
#         target_items[item_to_drop] = v_pref_last
#         sw[6].toc()
            
#         sw[7].tic()
#         loss_out = dropout_net.model.fit(
#             x = [u_pref_expanded[target_users, :], v_pref_expanded[target_items, :]] + user_content + item_content,
#             y = target_scores, # target
#             epochs = 1,
#             batch_size = data_batch_size,
#             shuffle = True
#         )
#         sw[7].toc()
#     dropout_model_path = '../model/dropout/model_{}.h5'.format(epoch)
#     dropout_net.model.save_weights(dropout_model_path)

Epoch 1/1
200000/200000 [==============================] - 26s 130us/step - loss: 0.0624
Epoch 1/1
200000/200000 [==============================] - 26s 130us/step - loss: 0.0593
Epoch 1/1
200000/200000 [==============================] - 26s 130us/step - loss: 0.0643
Epoch 1/1
200000/200000 [==============================] - 26s 130us/step - loss: 0.0624
Epoch 1/1
200000/200000 [==============================] - 26s 130us/step - loss: 0.0596
Epoch 1/1
200000/200000 [==============================] - 26s 129us/step - loss: 0.0619
Epoch 1/1
200000/200000 [==============================] - 26s 131us/step - loss: 0.0604
Epoch 1/1
200000/200000 [==============================] - 26s 132us/step - loss: 0.0599
Epoch 1/1
200000/200000 [==============================] - 26s 131us/step - loss: 0.0607
Epoch 1/1
200000/200000 [==============================] - 27s 134us/step - loss: 0.0630
Epoch 1/1
200000/200000 [==============================] - 27s 133us/step - loss: 0.0587
Epoch 1/1
200000/2000

200000/200000 [==============================] - 24s 121us/step - loss: 0.0568
Epoch 1/1
200000/200000 [==============================] - 26s 128us/step - loss: 0.0559
Epoch 1/1
200000/200000 [==============================] - 26s 128us/step - loss: 0.0558
Epoch 1/1
200000/200000 [==============================] - 26s 132us/step - loss: 0.0577
Epoch 1/1
200000/200000 [==============================] - 28s 138us/step - loss: 0.0568
Epoch 1/1
200000/200000 [==============================] - 28s 139us/step - loss: 0.0538
Epoch 1/1
200000/200000 [==============================] - 27s 134us/step - loss: 0.0601
Epoch 1/1
200000/200000 [==============================] - 26s 128us/step - loss: 0.0573
Epoch 1/1
200000/200000 [==============================] - 26s 128us/step - loss: 0.0584
Epoch 1/1
200000/200000 [==============================] - 26s 128us/step - loss: 0.0560
Epoch 1/1
200000/200000 [==============================] - 25s 127us/step - loss: 0.0543
Epoch 1/1
200000/200000 [======

200000/200000 [==============================] - 27s 137us/step - loss: 0.0555
Epoch 1/1
200000/200000 [==============================] - 27s 137us/step - loss: 0.0554
Epoch 1/1
200000/200000 [==============================] - 28s 139us/step - loss: 0.0547
Epoch 1/1
200000/200000 [==============================] - 29s 143us/step - loss: 0.0544
Epoch 1/1
200000/200000 [==============================] - 28s 142us/step - loss: 0.0556
Epoch 1/1
200000/200000 [==============================] - 29s 143us/step - loss: 0.0543
Epoch 1/1
200000/200000 [==============================] - 28s 142us/step - loss: 0.0530
Epoch 1/1
200000/200000 [==============================] - 28s 142us/step - loss: 0.0542
Epoch 1/1
200000/200000 [==============================] - 29s 144us/step - loss: 0.0537
Epoch 1/1
200000/200000 [==============================] - 29s 145us/step - loss: 0.0567
Epoch 1/1
200000/200000 [==============================] - 29s 143us/step - loss: 0.0545
Epoch 1/1
200000/200000 [======

In [35]:
# Evaluation
def recall_score_model(klist, data, v_user_all, v_item_all):
    recall_at_k = []
    max_k = max(klist)
    for user in data.user_list:
        # get the corresponding embedded vectors
        v_user = v_user_all[user]
        v_item = v_item_all[data.item_list[user]]
        
        # compute the scores
        score_list = np.matmul(v_user, v_item.T)
        score_list = score_list.flatten()
        # assert len(score_list) == len(data.item_list[user])
        
        k = min(max_k, len(data.item_list[user]))
        # get the recommended list
        indices = top_k(score_list, k)
        recommend_list = data.item_list[user][indices]
        
        # evaluate recall
        recall_at_k.append(__recall(klist, data.target_set[user], recommend_list))
    return np.mean(recall_at_k, axis=0)

In [38]:
sub_timer.start('evaluation of dropout net')
for model_num in range(10):
    model_path = '../model/dropout/exp2/model_%d.h5' % model_num
    dropout_net.model.load_weights(model_path)
    print('-'*40)
    print(model_path)
    user_content = generate_user_content(np.arange(num_user))
    item_content = generate_item_content(np.arange(num_item))
    dropout_user_all = dropout_net.user_model.predict([u_pref_scaled] + user_content)
    dropout_item_all = dropout_net.item_model.predict([v_pref_scaled] + item_content)

    klist = list(range(5, 51, 5))
    print('train')
    print(recall_random(klist, data_train))
    print(recall_score_model(klist, data_train, v_user_all, v_item_all))
    print(recall_score_model(klist, data_train, dropout_user_all, dropout_item_all))
    print()

    print('test warm')
    print(recall_random(klist, data_test_warm))
    print(recall_score_model(klist, data_test_warm, v_user_all, v_item_all))
    print(recall_score_model(klist, data_test_warm, dropout_user_all, dropout_item_all))
    print()
    
    print('test cold user')
    print(recall_random(klist, data_test_cold_user))
    print(recall_score_model(klist, data_test_cold_user, v_user_all, v_item_all))
    print(recall_score_model(klist, data_test_cold_user, dropout_user_all, dropout_item_all))
    print()

    print('test cold item')
    print(recall_random(klist, data_test_cold_item))
    print(recall_score_model(klist, data_test_cold_item, v_user_all, v_item_all))
    print(recall_score_model(klist, data_test_cold_item, dropout_user_all, dropout_item_all))
    print()
sub_timer.end()

[2018-05-01 21:10:54] (start) evaluation of dropout net
----------------------------------------
../model/dropout/model_0.h5
train
[ 0.28383819  0.3133038   0.33459792  0.35280115  0.37224075  0.39188641
  0.41090602  0.43108989  0.4511209   0.47031803]
[ 0.31106296  0.35011179  0.38146012  0.40964138  0.4366865   0.46233263
  0.48666695  0.51054204  0.53342427  0.55493316]
[ 0.29477555  0.33092184  0.35809482  0.38176983  0.40547192  0.42836127
  0.45017148  0.47237953  0.49376972  0.51391898]
()
test warm
[ 0.20321533  0.42469638  0.60585988  0.7350022   0.82007146  0.87812279
  0.91584418  0.94080075  0.95760223  0.9684273 ]
[ 0.26710343  0.49271289  0.65882621  0.77231052  0.84637798  0.89566094
  0.92805651  0.94954364  0.96388396  0.97326312]
[ 0.2543738   0.47892089  0.64780681  0.76468168  0.84083147  0.89196753
  0.92520767  0.94755884  0.96241878  0.97219976]
()
test cold user
[ 0.5454723   0.6753892   0.75819269  0.81751877  0.86159656  0.89500356
  0.92066166  0.94120628  0

[ 0.31106296  0.35011179  0.38146012  0.40964138  0.4366865   0.46233263
  0.48666695  0.51054204  0.53342427  0.55493316]
[ 0.29644614  0.33303041  0.36046081  0.3841419   0.40782158  0.4309071
  0.45262732  0.4747596   0.49607107  0.51624582]
()
test warm
[ 0.20302771  0.42493423  0.60513211  0.73461258  0.81994577  0.87794146
  0.91568782  0.9407536   0.95742468  0.96846893]
[ 0.26710343  0.49271289  0.65882621  0.77231052  0.84637798  0.89566094
  0.92805651  0.94954364  0.96388396  0.97326312]
[ 0.25682111  0.48025277  0.64904007  0.76539671  0.84136058  0.89240668
  0.92576656  0.94791385  0.96262302  0.97232464]
()
test cold user
[ 0.54650708  0.67584952  0.75796356  0.81729712  0.86164055  0.89506289
  0.92083681  0.94136213  0.95606276  0.96701262]
[ 0.54652435  0.67637865  0.75882391  0.81771064  0.86201169  0.89518416
  0.92084417  0.94130998  0.95601549  0.96692114]
[ 0.55424186  0.6847099   0.76703994  0.82509855  0.8677767   0.89994317
  0.92470475  0.94430257  0.95826385